In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import librosa.display
import numpy as np
import librosa
import tensorflow as tf
import glob

### If you have been save the data, you don't have to preprocessing and save the data

In [2]:
drone_path = '../../data/drone/*.wav'
background_path = '../../data/background/*.wav'

drone_files = glob.glob(drone_path)
background_files = glob.glob(background_path)

In [3]:
CHUNK_SIZE = 8192
SR = 44100
N_MFCC = 16

In [4]:
def load(files, sr=44100):
    [raw, sr] = librosa.load(files[0], sr=sr)
    for f in files[1:]:
        [array, sr] = librosa.load(f, sr=sr)
        raw = np.hstack((raw, array))
    print(raw.shape)
    return raw

In [5]:
drone_raw = load(drone_files)
background_raw = load(background_files)

(9583157,)
(20183327,)


# Data preprocessing

Chose `mfcc4` among functions below:
- input size 16x16; `n_mfcc=16`, used first 16 data points from 16x17 mfcc of a chunk with size 8192
- chunk siez 8192, overlapped half of it

In [6]:
def mfcc4(raw, label, chunk_size=8192, window_size=4096, sr=44100, n_mfcc=16, n_frame=16):
    mfcc = np.empty((0, n_mfcc, n_frame))
    y = []
    print(raw.shape)
    for i in range(0, len(raw), chunk_size//2):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc) #n_mfcc,17
        if mfcc_slice.shape[1] < 17:
            print("small end:", mfcc_slice.shape)
            continue
        mfcc_slice = mfcc_slice[:,:-1]
        mfcc_slice = mfcc_slice.reshape((1, mfcc_slice.shape[0], mfcc_slice.shape[1]))
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [7]:
mfcc_drone, y_drone = mfcc4(drone_raw, 1)
mfcc_background, y_background = mfcc4(background_raw, 0)

print(mfcc_drone.shape, y_drone.shape)
print(mfcc_background.shape, y_background.shape)

(9583157,)
small end: (16, 14)
small end: (16, 6)
(20183327,)
small end: (16, 13)
small end: (16, 5)
(2338, 16, 16) (2338,)
(4926, 16, 16) (4926,)


In [8]:
X = np.concatenate((mfcc_drone, mfcc_background), axis=0)
y = np.hstack((y_drone, y_background))
print(X.shape, y.shape)

(7264, 16, 16) (7264,)


In [9]:
X = np.reshape(X,(X.shape[0],-1))

In [10]:
X.shape

(7264, 256)

In [11]:
n_labels = y.shape[0]
n_unique_labels = 2
y_encoded = np.zeros((n_labels, n_unique_labels))
y_encoded[np.arange(n_labels), y] = 1
print(y_encoded.shape)

(7264, 2)


In [12]:
# Split data
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_encoded, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [13]:
print(X_train.shape,y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(4648, 256) (4648, 2)
(1163, 256) (1163, 2)
(1453, 256) (1453, 2)


In [14]:
# Save Data
np.save('../../model/X_train', X_train)
np.save('../../model/X_test', X_test)
np.save('../../model/X_val', X_val)
np.save('../../model/y_val', y_val)
np.save('../../model/y_train', y_train)
np.save('../../model/y_test', y_test)

### Until this part

In [15]:
# Load Data
X_train = np.load('../../model/X_train.npy')
X_test = np.load('../../model/X_test.npy')
X_val = np.load('../../model/X_val.npy')
y_val = np.load('../../model/y_val.npy')
y_train = np.load('../../model/y_train.npy')
y_test = np.load('../../model/y_test.npy')

# Experiment 3 - One convolutional layer /w no dropout


##Experiment 3-2
- learning rate 0.005
- pooling stride 1x1
- #filter 1
- best result among every other settings
- cost kept fluctuated during training. (0.8 -> 1.3) -- why is that?

In [16]:
tf.reset_default_graph()

In [17]:
n_mfcc = 16
n_frame = 16
n_classes = 2
n_channels = 1

kernel_size = 3
stride = 1
pad = "SAME"

learning_rate = 0.0002  # 0.005
training_epochs = 500 # 수정해야해

# Layer

## Experiment
- learning rate 0.0002
- no cnn/ pooling 
- just dense with dropout 

In [18]:
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
Y = tf.placeholder(tf.float32, shape=[None,n_classes])

keep_prob = tf.placeholder(tf.float32)

dense1 = tf.layers.dense(inputs=X, units=256, activation=tf.nn.relu)
dropout1 = tf.nn.dropout(dense1, keep_prob=keep_prob)
dense2 = tf.layers.dense(inputs=dropout1, units=256, activation=tf.nn.relu)
dropout2 = tf.nn.dropout(dense2, keep_prob=keep_prob)
dense3 = tf.layers.dense(inputs=dropout2, units=512, activation=tf.nn.relu)
dropout3 = tf.nn.dropout(dense3, keep_prob=keep_prob)
dense4 = tf.layers.dense(inputs=dropout3, units=512, activation=tf.nn.relu)
dropout4 = tf.nn.dropout(dense4, keep_prob=keep_prob)
dense5 = tf.layers.dense(inputs=dropout4, units=256, activation=tf.nn.relu)
dropout5 = tf.nn.dropout(dense5, keep_prob=keep_prob)

logits= tf.layers.dense(inputs=dropout5, units=2)

In [19]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [21]:
# model save
model_path = '../../model/Dense/dense_model'
saver = tf.train.Saver()

# Trainning

In [22]:
import math

In [23]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
#    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
#original 
batch_size = 32
cost_history = np.empty(shape=[1], dtype=float)

for epoch in range(1,500):
    avg_cost = 0    
#    total_batch = int(X_train.shape[0] / batch_size)
    for i in range( int(math.ceil(len(X_train)/batch_size)) ): # 배치 사이즈로 나눈 나머지 다 돌 수 있게 
        x_ = X_train[batch_size * i: batch_size * (i + 1)]
        y_ = y_train[batch_size * i: batch_size * (i + 1)]
        feed_dict={X:x_, Y:y_, keep_prob:0.6}    
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        cost_history = np.append(cost_history,cost)
        avg_cost += c
    #accuracy_val = sess.run([accuracy], feed_dict={X:X_val, Y:y_val
    val_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_val , keep_prob:1})  
    val_true = sess.run(tf.argmax(y_val,1))
    accuracy_val = accuracy_score(val_pred, val_true)
    print('Epoch:', '%04d' % (epoch), 'cost = ', '{:.9f}'.format(avg_cost / len(X_train)), "\t검증 세트 정확도: {:.3f}%".format(accuracy_val * 100),)
saver.save(sess, model_path)

#if epoch% 20 == 0:
#   save

Epoch: 0001 cost =  0.439514566 	검증 세트 정확도: 91.917%
Epoch: 0002 cost =  0.108595408 	검증 세트 정확도: 89.940%
Epoch: 0003 cost =  0.052662655 	검증 세트 정확도: 89.338%
Epoch: 0004 cost =  0.030311324 	검증 세트 정확도: 91.660%
Epoch: 0005 cost =  0.020421221 	검증 세트 정확도: 92.949%
Epoch: 0006 cost =  0.014987682 	검증 세트 정확도: 94.497%
Epoch: 0007 cost =  0.010725190 	검증 세트 정확도: 96.733%
Epoch: 0008 cost =  0.007928660 	검증 세트 정확도: 97.334%
Epoch: 0009 cost =  0.006188294 	검증 세트 정확도: 97.936%
Epoch: 0010 cost =  0.004175699 	검증 세트 정확도: 98.366%
Epoch: 0011 cost =  0.004263082 	검증 세트 정확도: 98.366%
Epoch: 0012 cost =  0.002861895 	검증 세트 정확도: 98.624%
Epoch: 0013 cost =  0.002720254 	검증 세트 정확도: 98.796%
Epoch: 0014 cost =  0.002012771 	검증 세트 정확도: 98.624%
Epoch: 0015 cost =  0.002103205 	검증 세트 정확도: 99.054%
Epoch: 0016 cost =  0.001960866 	검증 세트 정확도: 99.054%
Epoch: 0017 cost =  0.001481497 	검증 세트 정확도: 98.968%
Epoch: 0018 cost =  0.001212291 	검증 세트 정확도: 99.054%
Epoch: 0019 cost =  0.001492842 	검증 세트 정확도: 98.968%
Epoch: 0020 

Epoch: 0159 cost =  0.000198458 	검증 세트 정확도: 99.054%
Epoch: 0160 cost =  0.000146296 	검증 세트 정확도: 99.656%
Epoch: 0161 cost =  0.000053573 	검증 세트 정확도: 99.656%
Epoch: 0162 cost =  0.000227716 	검증 세트 정확도: 99.398%
Epoch: 0163 cost =  0.000163900 	검증 세트 정확도: 99.656%
Epoch: 0164 cost =  0.000019287 	검증 세트 정확도: 99.656%
Epoch: 0165 cost =  0.000071837 	검증 세트 정확도: 99.570%
Epoch: 0166 cost =  0.000309314 	검증 세트 정확도: 99.398%
Epoch: 0167 cost =  0.000364036 	검증 세트 정확도: 99.484%
Epoch: 0168 cost =  0.000109801 	검증 세트 정확도: 99.656%
Epoch: 0169 cost =  0.000354163 	검증 세트 정확도: 99.570%
Epoch: 0170 cost =  0.000194230 	검증 세트 정확도: 99.656%
Epoch: 0171 cost =  0.000028367 	검증 세트 정확도: 99.570%
Epoch: 0172 cost =  0.000271245 	검증 세트 정확도: 99.484%
Epoch: 0173 cost =  0.000279318 	검증 세트 정확도: 99.656%
Epoch: 0174 cost =  0.000240847 	검증 세트 정확도: 99.484%
Epoch: 0175 cost =  0.000251076 	검증 세트 정확도: 99.570%
Epoch: 0176 cost =  0.000228193 	검증 세트 정확도: 99.570%
Epoch: 0177 cost =  0.000027131 	검증 세트 정확도: 99.570%
Epoch: 0178 

Epoch: 0317 cost =  0.000159819 	검증 세트 정확도: 99.484%
Epoch: 0318 cost =  0.000301226 	검증 세트 정확도: 99.570%
Epoch: 0319 cost =  0.000064199 	검증 세트 정확도: 99.484%
Epoch: 0320 cost =  0.000449383 	검증 세트 정확도: 99.570%
Epoch: 0321 cost =  0.000036114 	검증 세트 정확도: 99.656%
Epoch: 0322 cost =  0.000045075 	검증 세트 정확도: 99.570%
Epoch: 0323 cost =  0.000027536 	검증 세트 정확도: 99.570%
Epoch: 0324 cost =  0.000014656 	검증 세트 정확도: 99.570%
Epoch: 0325 cost =  0.000070818 	검증 세트 정확도: 99.570%
Epoch: 0326 cost =  0.000064196 	검증 세트 정확도: 99.742%
Epoch: 0327 cost =  0.000032736 	검증 세트 정확도: 99.484%
Epoch: 0328 cost =  0.000635986 	검증 세트 정확도: 99.742%
Epoch: 0329 cost =  0.000023255 	검증 세트 정확도: 99.742%
Epoch: 0330 cost =  0.000076188 	검증 세트 정확도: 99.742%
Epoch: 0331 cost =  0.000338535 	검증 세트 정확도: 99.656%
Epoch: 0332 cost =  0.000238233 	검증 세트 정확도: 99.484%
Epoch: 0333 cost =  0.000190662 	검증 세트 정확도: 99.656%
Epoch: 0334 cost =  0.000121955 	검증 세트 정확도: 99.570%
Epoch: 0335 cost =  0.000117273 	검증 세트 정확도: 99.656%
Epoch: 0336 

Epoch: 0475 cost =  0.000048057 	검증 세트 정확도: 99.570%
Epoch: 0476 cost =  0.000189726 	검증 세트 정확도: 99.656%
Epoch: 0477 cost =  0.000140855 	검증 세트 정확도: 99.570%
Epoch: 0478 cost =  0.000118730 	검증 세트 정확도: 99.742%
Epoch: 0479 cost =  0.000086605 	검증 세트 정확도: 99.742%
Epoch: 0480 cost =  0.000012091 	검증 세트 정확도: 99.828%
Epoch: 0481 cost =  0.000034802 	검증 세트 정확도: 99.742%
Epoch: 0482 cost =  0.000069259 	검증 세트 정확도: 99.656%
Epoch: 0483 cost =  0.000020414 	검증 세트 정확도: 99.742%
Epoch: 0484 cost =  0.000117419 	검증 세트 정확도: 99.742%
Epoch: 0485 cost =  0.000096330 	검증 세트 정확도: 99.570%
Epoch: 0486 cost =  0.000100897 	검증 세트 정확도: 99.656%
Epoch: 0487 cost =  0.000130173 	검증 세트 정확도: 99.742%
Epoch: 0488 cost =  0.000217453 	검증 세트 정확도: 99.742%
Epoch: 0489 cost =  0.000101100 	검증 세트 정확도: 99.656%
Epoch: 0490 cost =  0.000185155 	검증 세트 정확도: 99.656%
Epoch: 0491 cost =  0.000201539 	검증 세트 정확도: 99.742%
Epoch: 0492 cost =  0.000027266 	검증 세트 정확도: 99.742%
Epoch: 0493 cost =  0.000022416 	검증 세트 정확도: 99.742%
Epoch: 0494 

'../../model/Dense/dense_model'

## Prediction

In [26]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test , keep_prob:1}) 
y_true = sess.run(tf.argmax(y_test,1))

In [27]:
# Ptint Result

from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.999
Accuracy:  0.9986235375086029
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       966
           1       1.00      1.00      1.00       487

   micro avg       1.00      1.00      1.00      1453
   macro avg       1.00      1.00      1.00      1453
weighted avg       1.00      1.00      1.00      1453

[[966   0]
 [  2 485]]
